In [4]:
import numpy as np
#import cupy as cp
import matplotlib.pyplot as plt
import cv2
from scipy import stats

PIC1D open-boundaryのテスト

In [74]:
def get_rho(q_list, x, n_x, dx, rho):

    x_index = np.floor(x[0, :] / dx).astype(np.int64)

    cx1 = (x[0, :] - x_index*dx)/dx  
    cx2 = ((x_index+1)*dx - x[0, :])/dx 
    index_one_array = x_index

    rho += np.bincount(index_one_array, 
                       weights=q_list * cx2,
                       minlength=n_x
                      )
    rho += np.roll(np.bincount(index_one_array, 
                               weights=q_list * cx1,
                               minlength=n_x
                              ), 1, axis=0)
    
    return rho


def get_current_density(c, q_list, v, x, n_x, dx, dt, current):

    x_index = np.floor(x[0, :] / dx).astype(int)
    x_index_half = np.floor((x[0, :] - 1/2*dx) / dx).astype(int)
    x_index_half_minus = np.where(x_index_half == -1)
    x_index_half[x_index_half == -1] = n_x-1

    gamma = np.sqrt(1.0 + (np.linalg.norm(v, axis=0)/c)**2)

    cx1 = (x[0, :] - (x_index_half + 1/2)*dx)/dx  
    cx2 = ((x_index_half + 3/2)*dx - x[0, :])/dx 
    cx1[x_index_half_minus] = (x[0, x_index_half_minus] - (-1/2)*dx)/dx  
    cx2[x_index_half_minus] = ((1/2)*dx - x[0, x_index_half_minus])/dx 
    index_one_array = x_index_half

    current[0, :] += np.bincount(index_one_array, 
                                 weights=q_list * v[0, :]/gamma * cx2, 
                                 minlength=n_x
                                )
    current[0, :] += np.roll(np.bincount(index_one_array, 
                                         weights=q_list * v[0, :]/gamma * cx1, 
                                         minlength=n_x
                                        ), 1, axis=0)


    cx1 = (x[0, :] - x_index*dx)/dx  
    cx2 = ((x_index+1)*dx - x[0, :])/dx 
    index_one_array = x_index
    
    current[1, :] += np.bincount(index_one_array, 
                                    weights=q_list * v[1, :]/gamma * cx2, 
                                    minlength=n_x
                                    )
    current[1, :] += np.roll(np.bincount(index_one_array, 
                                            weights=q_list * v[1, :]/gamma * cx1, 
                                            minlength=n_x
                                            ), 1, axis=0)

    current[2, :] += np.bincount(index_one_array, 
                                 weights=q_list * v[2, :]/gamma * cx2, 
                                 minlength=n_x
                                )
    current[2, :] += np.roll(np.bincount(index_one_array, 
                                         weights=q_list * v[2, :]/gamma * cx1, 
                                         minlength=n_x
                                        ), 1, axis=0)
    
    return current


def buneman_boris_v(c, dt, q_list, m_list, E, B, v):

    gamma = np.sqrt(1.0 + (np.linalg.norm(v, axis=0)/c)**2)

    #TとSの設定
    T = (q_list/m_list) * dt * B / 2.0 / gamma
    S = 2.0 * T / (1.0 + np.linalg.norm(T, axis=0)**2)

    #時間発展
    v_minus = v + (q_list/m_list) * E * (dt/2)
    v_0 = v_minus + np.cross(v_minus, T, axis=0)
    v_plus = v_minus + np.cross(v_0, S, axis=0)
    v = v_plus + (q_list/m_list) * E * (dt/2.0)

    return v 


def buneman_boris_x(c, dt, v, x):

    gamma = np.sqrt(1.0 + (np.linalg.norm(v, axis=0)/c)**2)

    x = x + v * dt / gamma

    return x


def time_evolution_v(c, E, B, x, q_list, m_list, n_x, dx, dt, v):
    E_tmp = E.copy()
    B_tmp = B.copy()
    E_tmp[0, :] = (E[0, :] + np.roll(E[0, :], 1, axis=0)) / 2.0
    B_tmp[0, :] = (B[0, :] + np.roll(B[0, :], -1, axis=0)) / 2.0
    x_index = np.floor(x[0, :] / dx).astype(int)
    x_index_half = np.floor((x[0, :] - 1/2*dx) / dx).astype(int)
    x_index_half_minus = np.where(x_index_half == -1)
    x_index_half[x_index_half == -1] = n_x-1
    E_particle = np.zeros(x.shape)
    B_particle = np.zeros(x.shape)

    #電場
    cx1 = (x[0, :] - x_index*dx)/dx  
    cx2 = ((x_index+1)*dx - x[0, :])/dx 
    cx1 = cx1.reshape(-1, 1)
    cx2 = cx2.reshape(-1, 1)
    E_particle[:, :] = (E_tmp[:, x_index].T * cx2 \
                     + E_tmp[:, (x_index+1)%n_x].T * cx1 \
                    ).T
    
    #磁場
    cx1 = (x[0, :] - (x_index_half + 1/2)*dx)/dx  
    cx2 = ((x_index_half + 3/2)*dx - x[0, :])/dx 
    cx1[x_index_half_minus] = (x[0, x_index_half_minus] - (-1/2)*dx)/dx  
    cx2[x_index_half_minus] = ((1/2)*dx - x[0, x_index_half_minus])/dx 
    cx1 = cx1.reshape(-1, 1)
    cx2 = cx2.reshape(-1, 1)
    B_particle[:, :] = (B_tmp[:, x_index_half].T * cx2 \
                     + B_tmp[:, (x_index_half+1)%n_x].T * cx1 \
                    ).T
  
    v = buneman_boris_v(c, dt, q_list, m_list, E_particle, B_particle, v)

    return v


def time_evolution_x(c, dt, v, x):
    
    x = buneman_boris_x(c, dt, v, x)

    return x 


def time_evolution_E(B, current, c, epsilon0, dx, dt, E):
    E[0, :] = -current[0, :]/epsilon0 * dt + E[0, :]
    E[1, :] = (-current[1, :]/epsilon0 \
            - c**2 * (B[2, :] - np.roll(B[2, :], 1))/dx) * dt \
            + E[1, :]
    E[2, :] = (-current[2, :]/epsilon0 \
            + c**2 * (B[1, :] - np.roll(B[1, :], 1)) / dx) * dt \
            + E[2, :]
    return E


def time_evolution_B(E, dx, dt, B):
    B[0, :] = B[0, :]
    B[1, :] = -(-(np.roll(E[2, :], -1) - E[2, :]) / dx) * dt \
            + B[1, :]
    B[2, :] = -((np.roll(E[1, :], -1) - E[1, :]) / dx) * dt \
            + B[2, :]
    return B


def periodic_condition_x(x, x_max):

    over_xmax_index = np.where(x[0, :] >= x_max)[0]
    x[0, over_xmax_index] = 1e-10

    under_x0_index = np.where(x[0, :] <= 0.0)[0]
    x[0, under_x0_index] = x_max - 1e-10

    return x 


def refrective_condition_x(v, x, x_max):

    over_xmax_index = np.where(x[0, :] >= x_max)[0]
    x[0, over_xmax_index] = x_max - 1e-10
    v[0, over_xmax_index] = -v[0, over_xmax_index]

    under_x0_index = np.where(x[0, :] <= 0.0)[0]
    x[0, under_x0_index] = 1e-10
    v[0, under_x0_index] = -v[0, under_x0_index]

    return v, x


def open_condition_x(v_past, x_past, x_min, x_max, dx, v, x):

    first_inf_index = np.argmax(v[0, :] == np.inf) #v[0]でなくてもOK
    
    new_particle_x0_index = np.where((x_past[0, :] < x_min + dx) & (x[0, :] >= x_min + dx))[0]
    num_new_particles = len(new_particle_x0_index)
    v[:, first_inf_index:first_inf_index+num_new_particles] = v[:, new_particle_x0_index]
    x[0, first_inf_index:first_inf_index+num_new_particles] = x[0, new_particle_x0_index] - dx

    first_inf_index += num_new_particles
    new_particle_xmax_index = np.where((x_past[0, :] > x_max - dx) & (x[0, :] <= x_max - dx))[0]
    num_new_particles = len(new_particle_xmax_index)
    v[:, first_inf_index:first_inf_index+num_new_particles] = v[:, new_particle_xmax_index]
    x[0, first_inf_index:first_inf_index+num_new_particles] = x[0, new_particle_xmax_index] + dx


    over_xmax_index = np.where(x[0, :] >= x_max)[0]
    v[:, over_xmax_index] = np.inf
    x[:, over_xmax_index] = np.inf

    under_x0_index = np.where(x[0, :] <= x_min)[0]
    v[:, under_x0_index] = np.inf
    x[:, under_x0_index] = np.inf

    v, x = sort_particles(v, x)

    return v, x


def sort_particles(v, x):

    exist_particles_index = np.isfinite(v[0, :]) #v[0]でなくてもOK
    v[0] = np.concatenate(
        (v[0, exist_particles_index], 
         [np.inf] * (v.shape[1] - exist_particles_index.sum())
        ))
    v[1] = np.concatenate(
        (v[0, exist_particles_index], 
         [np.inf] * (v.shape[1] - exist_particles_index.sum())
        ))
    v[2] = np.concatenate(
        (v[0, exist_particles_index], 
         [np.inf] * (v.shape[1] - exist_particles_index.sum())
        ))
    x[0] = np.concatenate(
        (x[0, exist_particles_index], 
         [np.inf] * (x.shape[1] - exist_particles_index.sum())
        ))
    x[1] = np.concatenate(
        (x[0, exist_particles_index], 
         [np.inf] * (x.shape[1] - exist_particles_index.sum())
        ))
    x[2] = np.concatenate(
        (x[0, exist_particles_index], 
         [np.inf] * (x.shape[1] - exist_particles_index.sum())
        ))
    
    return v, x

初期設定

In [102]:
c = 0.5
epsilon0 = 1.0
mu_0 = 1.0 / (epsilon0 * c**2)
m_unit = 1.0
r_m = 1/100
t_r = 1/100
m_electron = 1 * m_unit
m_ion = m_electron / r_m
r_q = 1.0
T_e = 1/2 * m_electron * (0.01*c)**2
T_i = T_e / t_r
n_e = 100 #ここは手動で調整すること
B0 = np.sqrt(n_e) / 10.0
q_unit = np.sqrt(epsilon0 * T_e / n_e)
q_electron = -1 * q_unit
q_ion = r_q * q_unit
n_i = int(n_e * np.abs(q_electron) / q_ion)
omega_pe = np.sqrt(n_e * q_electron**2 / m_electron / epsilon0)
omega_pi = np.sqrt(n_i * q_ion**2 / m_ion / epsilon0)
omega_ce = q_electron * B0 / m_electron
omega_ci = q_ion * B0 / m_ion
V_A = c * np.sqrt(B0**2 / (n_e*m_electron + n_i*m_ion))
C_S = np.sqrt(r_m * T_e)
debye_length = np.sqrt(epsilon0 * T_e / n_e / q_electron**2)

dx = debye_length
n_x = 512
x_min = 0.0
x_max = n_x * dx
x_coordinate = np.arange(0.0, x_max, dx)
dt = 1.0
step = 50000
t_max = step * dt
v_thermal_ion = np.sqrt(T_i / m_ion)
v_thermal_electron = np.sqrt(T_e / m_electron)
v_ion = np.array([0.0, 0.0, 0.0])
v_electron = np.array([-10.0*v_thermal_ion, 0.0, 0.0])
v_beam = np.array([10.0*v_thermal_ion, 0.0, 0.0])
if c * dt > dx:
    print(f"You had better change some parameters! \nCFL condition is not satisfied \n c * dt = {c * dt} > dx = {dx} \n")
else:
    print(f'c * dt = {c * dt} < dx = {dx} \n')

if omega_pe * dt > 0.2:
    print(f"You had better change some parameters! \n$\omega$_pe * dt = {omega_pe * dt} > 0.1 \n")
if round(dx, 5) != 1.0:
    print(f"You had better change some parameters! \ndebye length = {debye_length} should be equal to grid size = {dx} \n")


E = np.zeros([3, len(x_coordinate)])
B = np.zeros([3, len(x_coordinate)])
current = np.zeros([3, len(x_coordinate)])

n_ion = int(n_x * n_i)
n_electron = int(n_x * n_e / 2)
n_beam = int(n_x * n_e / 2)
n_buffer_ion = int(n_i * n_x / 5)
n_buffer_electron = int(n_i * n_x / 5)
np.random.RandomState(1)
x_start_ion = np.random.rand(n_ion) * x_max
x_start_electron = np.random.rand(n_electron) * x_max
x_start_beam = np.random.rand(n_beam) * x_max
n_total = n_ion + n_buffer_ion + n_electron + n_beam + n_buffer_electron
print(f"total number of particle is {n_total}")

x = np.zeros([3, n_total])
x[0, :n_ion] = x_start_ion
x[0, int(n_total/2):int(n_total/2) + n_electron] = x_start_electron
x[0, int(n_total/2) + n_electron:int(n_total/2) + n_electron + n_beam] = x_start_beam
x[:, n_ion:int(n_total/2)] = np.inf
x[:, int(n_total/2) + n_electron + n_beam:] = np.inf

v = np.zeros([3, n_total])
v[0, :n_ion] = np.array(stats.norm.rvs(v_ion[0], v_thermal_ion, size=n_ion))
v[0, int(n_total/2):int(n_total/2) + n_electron] = np.array(stats.norm.rvs(v_electron[0], v_thermal_electron, size=n_electron))
v[0, int(n_total/2) + n_electron:int(n_total/2) + n_electron + n_beam] = np.array(stats.norm.rvs(v_beam[0], v_thermal_electron, size=n_beam))
v[1, :n_ion] = np.array(stats.norm.rvs(v_ion[1], v_thermal_ion, size=n_ion))
v[1, int(n_total/2):int(n_total/2) + n_electron] = np.array(stats.norm.rvs(v_electron[1], v_thermal_electron, size=n_electron))
v[1, int(n_total/2) + n_electron:int(n_total/2) + n_electron + n_beam] = np.array(stats.norm.rvs(v_beam[1], v_thermal_electron, size=n_beam))
v[2, :n_ion] = np.array(stats.norm.rvs(v_ion[2], v_thermal_ion, size=n_ion))
v[2, int(n_total/2):int(n_total/2) + n_electron] = np.array(stats.norm.rvs(v_electron[2], v_thermal_electron, size=n_electron))
v[2, int(n_total/2) + n_electron:int(n_total/2) + n_electron + n_beam] = np.array(stats.norm.rvs(v_beam[2], v_thermal_electron, size=n_beam))
v[:, n_ion:int(n_total/2)] = np.inf
v[:, int(n_total/2) + n_electron + n_beam:] = np.inf

q_list = np.zeros(n_total)
q_list[:n_ion] = q_ion
q_list[int(n_total/2):int(n_total/2) + n_electron + n_beam] = q_electron
m_list = np.zeros(n_total)
m_list[:n_ion] = m_ion
m_list[int(n_total/2):int(n_total/2) + n_electron + n_beam] = m_electron

c * dt = 0.5 < dx = 1.0 

total number of particle is 122880


In [103]:
#STEP1
#rho = get_rho(q_list, x, n_x, dx)
#E = solve_poisson_not_periodic(rho, n_x, epsilon0, dx, E)  

first_inf_index_ion = np.argmax(v[0, :int(n_total/2)] == np.inf) #v[0]でなくてもOK
first_inf_index_electron = np.argmax(v[0, int(n_total/2):] == np.inf) \
                         + int(n_total/2) #v[0]でなくてもOK

for k in range(step+1):
    if k % 100 == 0:
        print(f'{k*dt} done...')

    #STEP2
    B = time_evolution_B(E, dx, dt/2, B)   
    #STEP3
    v[:, :first_inf_index_ion] = time_evolution_v(
        c, E, B, x[:, :first_inf_index_ion], 
        q_list[:first_inf_index_ion], 
        m_list[:first_inf_index_ion], 
        n_x, dx, dt, v[:, :first_inf_index_ion]
        )
    v[:, int(n_total/2):first_inf_index_electron] = time_evolution_v(
        c, E, B, x[:, int(n_total/2):first_inf_index_electron], 
        q_list[int(n_total/2):first_inf_index_electron], 
        m_list[int(n_total/2):first_inf_index_electron], 
        n_x, dx, dt, v[:, int(n_total/2):first_inf_index_electron]
        )
    #STEP4
    v_past = v.copy()
    x_past = x.copy()
    x[:, :first_inf_index_ion] = time_evolution_x(
        c, dt/2, v[:, :first_inf_index_ion], 
        x[:, :first_inf_index_ion]
        )
    x[:, int(n_total/2):first_inf_index_electron] = time_evolution_x(
        c, dt/2, v[:, int(n_total/2):first_inf_index_electron], 
        x[:, int(n_total/2):first_inf_index_electron]
        )
    v[:, :int(n_total/2)], x[:, :int(n_total/2)] = open_condition_x(
        v_past[:, :int(n_total/2)], x_past[:, :int(n_total/2)], 
        x_min, x_max, dx, 
        v[:, :int(n_total/2)], x[:, :int(n_total/2)])
    v[:, int(n_total/2):], x[:, int(n_total/2):] = open_condition_x(
        v_past[:, int(n_total/2):], x_past[:, int(n_total/2):], 
        x_min, x_max, dx, 
        v[:, int(n_total/2):], x[:, int(n_total/2):])
    first_inf_index_ion = np.argmax(v[0, :int(n_total/2)] == np.inf) #v[0]でなくてもOK
    first_inf_index_electron = np.argmax(v[0, int(n_total/2):] == np.inf) \
                            + int(n_total/2) #v[0]でなくてもOK
    #STEP5
    current = np.zeros([3, n_x])
    current = get_current_density(c, q_list[:first_inf_index_ion], 
                                  v[:, :first_inf_index_ion], 
                                  x[:, :first_inf_index_ion], 
                                  n_x, dx, dt, current)
    current = get_current_density(c, q_list[int(n_total/2):first_inf_index_electron], 
                                  v[:, int(n_total/2):first_inf_index_electron], 
                                  x[:, int(n_total/2):first_inf_index_electron], 
                                  n_x, dx, dt, current)
    #STEP6
    B = time_evolution_B(E, dx, dt/2, B) 
    #STEP7
    E = time_evolution_E(B, current, c, epsilon0, dx, dt, E)
    #STEP8
    v_past = v.copy()
    x_past = x.copy()
    x = time_evolution_x(c, dt/2, v, x)
    v[:, :int(n_total/2)], x[:, :int(n_total/2)] = open_condition_x(
        v_past[:, :int(n_total/2)], x_past[:, :int(n_total/2)], 
        x_min, x_max, dx, 
        v[:, :int(n_total/2)], x[:, :int(n_total/2)])
    v[:, int(n_total/2):], x[:, int(n_total/2):] = open_condition_x(
        v_past[:, int(n_total/2):], x_past[:, int(n_total/2):], 
        x_min, x_max, dx, 
        v[:, int(n_total/2):], x[:, int(n_total/2):])
    first_inf_index_ion = np.argmax(v[0, :int(n_total/2)] == np.inf) #v[0]でなくてもOK
    first_inf_index_electron = np.argmax(v[0, int(n_total/2):] == np.inf) \
                            + int(n_total/2) #v[0]でなくてもOK

    #if k % 100 == 0:
    #    k1 = k // 100
    #    first_inf_index = np.argmax(v[0, :] == np.inf) #v[0]でなくてもOK
    #    KE = np.sum(1/2 * m_list * np.linalg.norm(v[:, :first_inf_index], axis=0)**2)
    #    np.save(f'two_stream_electron_open/results_1pic_two_stream_electron_xv_{k1}.npy', np.concatenate([x[:, :first_inf_index], v[:, :first_inf_index]]))
    #    np.save(f'two_stream_electron_open/results_1pic_two_stream_electron_E_{k1}.npy', E)
    #    np.save(f'two_stream_electron_open/results_1pic_two_stream_electron_B_{k1}.npy', B)
    #    np.save(f'two_stream_electron_open/results_1pic_two_stream_electron_current_{k1}.npy', current)
    #    np.save(f'two_stream_electron_open/results_1pic_two_stream_electron_KE_{k1}.npy', KE)

[ 68.24453248 472.51795463 385.35091084 ...          inf          inf
          inf]
0.0 done...
[ 68.24765021 472.52313778 385.35100743 ...          inf          inf
          inf]
[ 68.25076794 472.52832083 385.35110402 ...          inf          inf
          inf]


C:\Users\keita\AppData\Local\Temp\ipykernel_12064\3669378286.py:51: RuntimeWarning: invalid value encountered in multiply
  weights=q_list * v[1, :]/gamma * cx2,
C:\Users\keita\AppData\Local\Temp\ipykernel_12064\3669378286.py:51: RuntimeWarning: invalid value encountered in true_divide
  weights=q_list * v[1, :]/gamma * cx2,
C:\Users\keita\AppData\Local\Temp\ipykernel_12064\3669378286.py:55: RuntimeWarning: invalid value encountered in multiply
  weights=q_list * v[1, :]/gamma * cx1,
C:\Users\keita\AppData\Local\Temp\ipykernel_12064\3669378286.py:55: RuntimeWarning: invalid value encountered in true_divide
  weights=q_list * v[1, :]/gamma * cx1,
C:\Users\keita\AppData\Local\Temp\ipykernel_12064\3669378286.py:60: RuntimeWarning: invalid value encountered in multiply
  weights=q_list * v[2, :]/gamma * cx2,
C:\Users\keita\AppData\Local\Temp\ipykernel_12064\3669378286.py:60: RuntimeWarning: invalid value encountered in true_divide
  weights=q_list * v[2, :]/gamma * cx2,
C:\Users\keita\AppD

[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf inf ... inf inf inf]
[inf inf i

KeyboardInterrupt: 

プロット

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111)

results_total = np.load(f"./two_stream_electron/results_1pic_two_stream_electron_xv_{25}.npy")
ax.scatter(results_total[0, :n_plus], results_total[3, :n_plus]/v_thermal_ion, s=0.1, c='orange', label='ion')
ax.scatter(results_total[0, n_plus:n_plus+n_minus], results_total[3, n_plus:n_plus+n_minus]/v_thermal_electron, s=0.1, c='blue', label='electron(background)')
ax.scatter(results_total[0, n_plus+n_minus:], results_total[3, n_plus+n_minus:]/v_thermal_electron, s=0.1, c='green', marker='*', label='electron(beam)')
ax.set_xlim(0, x_max)
ax.set_ylim(-30, 30)
ax.set_xlabel('$x / \lambda_D$', fontsize=20)
ax.set_ylabel('$V_x / V_{thi}$', fontsize=20)
ax.tick_params(labelsize=18)
ax.legend(loc='upper right', fontsize=20)
    

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111)

interval = 1
number_of_results = 501
for i in range(number_of_results):
    results_total = np.load(f"./two_stream_electron/results_1pic_two_stream_electron_xv_{i}.npy")
    ax.clear()
    ax.scatter(results_total[0, :n_plus], results_total[3, :n_plus]/v_thermal_ion, s=0.1, c='orange', label='ion')
    ax.scatter(results_total[0, n_plus:n_plus+n_minus], results_total[3, n_plus:n_plus+n_minus]/v_thermal_electron, s=0.1, c='blue', label='electron(background)')
    ax.scatter(results_total[0, n_plus+n_minus:], results_total[3, n_plus+n_minus:]/v_thermal_electron, s=0.1, c='green', marker='*', label='electron(beam)')
    ax.set_xlim(0, x_max)
    ax.set_ylim(-30, 30)
    ax.set_xlabel('$x / \lambda_D$', fontsize=20)
    ax.set_ylabel('$V_x / V_{thi}$', fontsize=20)
    ax.tick_params(labelsize=18)
    ax.legend(loc='upper right', fontsize=20)
    fig.savefig(f'./pictures/{i}.png', dpi=200)


img = cv2.imread(f'./pictures/0.png')
framesize = (img.shape[1], img.shape[0])
fps = 10.0

fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
video  = cv2.VideoWriter(filename=f'two_stream_electron.mp4', fourcc=fourcc, fps=fps, frameSize=framesize)

for i in range(number_of_results):
    img = cv2.imread(f'./pictures/{i}.png')
    video.write(img)
video.release()